# Data 전처리 과정

크롤링 과정을 통해 2022.01.11 기준 스팀게임 탑100의 현재까지의 방문자 수 데이터를 긁어왔다.  
이를 바탕으로, 시각화 리포트를 만들고 유저의 흐름에 따라 게임군을 나눠서 이에 대해 인사이트를 얻어보는 것이 목표이다.  

이번에는 sql등을 활용하기 위해 우선적으로는 데이터를 전처리하고자 한다.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./steam_game_data.csv', index_col = 0)

In [3]:
data

,Game,Month,Avg_Players,Peak_Players
0,\n\n\t\t\t\t\t\tCounter-Strike: Global Offensi...,\n\t\t\t\t\tDecember 2021\n\t\t\t\t,546614.19,950586
1,\n\n\t\t\t\t\t\tCounter-Strike: Global Offensi...,\n\t\t\t\t\tNovember 2021\n\t\t\t\t,548161.67,935593
2,\n\n\t\t\t\t\t\tCounter-Strike: Global Offensi...,\n\t\t\t\t\tOctober 2021\n\t\t\t\t,512435.85,864966
3,\n\n\t\t\t\t\t\tCounter-Strike: Global Offensi...,\n\t\t\t\t\tSeptember 2021\n\t\t\t\t,512350.92,942519
4,\n\n\t\t\t\t\t\tCounter-Strike: Global Offensi...,\n\t\t\t\t\tAugust 2021\n\t\t\t\t,512081.96,802544
...,...,...,...,...
5424,\n\n\t\t\t\t\t\tHollow Knight\n\t\t\t\t\t\n,\n\t\t\t\t\tApril 2017\n\t\t\t\t,728.86,1305
5425,\n\n\t\t\t\t\t\tHollow Knight\n\t\t\t\t\t\n,\n\t\t\t\t\tMarch 2017\n\t\t\t\t,1126.45,2027
5426,\n\n\t\t\t\t\t\tHollow Knight\n\t\t\t\t\t\n,\n\t\t\t\t\tFebruary 2017\n\t\t\t\t,192.30,2077
5427,\n\n\t\t\t\t\t\tHollow Knight\n\t\t\t\t\t\n,\n\t\t\t\t\tJanuary 2017\n\t\t\t\t,1.96,6


이런, 매우 더럽게 나왔다. 그 이유를 확인해보면, 표에 데이터가 들어가면서 공백 문자들이 들어간 것으로 보인다.

In [4]:
data.loc[550][0][8:-7]

'Destiny 2'

In [5]:
data.loc[0][1][6:-5]

'December 2021'

우선, \n와 \t를 제거하자.

In [6]:
data['Game'] = data.Game.apply(lambda s : s[8:-7])
data['Month'] = data.Month.apply(lambda s : s[6:-5])

In [7]:
data

,Game,Month,Avg_Players,Peak_Players
0,Counter-Strike: Global Offensive,December 2021,546614.19,950586
1,Counter-Strike: Global Offensive,November 2021,548161.67,935593
2,Counter-Strike: Global Offensive,October 2021,512435.85,864966
3,Counter-Strike: Global Offensive,September 2021,512350.92,942519
4,Counter-Strike: Global Offensive,August 2021,512081.96,802544
...,...,...,...,...
5424,Hollow Knight,April 2017,728.86,1305
5425,Hollow Knight,March 2017,1126.45,2027
5426,Hollow Knight,February 2017,192.30,2077
5427,Hollow Knight,January 2017,1.96,6


month를 잘 분리하는 것이중요할 것이다. 이를 잘 분리해보자.

In [8]:
year_dict = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
             'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

In [9]:
data['Year'] = data.Month.apply(lambda s : s[-4:])
data['Month'] = data.Month.apply(lambda s : year_dict[s[:-5]])

In [10]:
data

,Game,Month,Avg_Players,Peak_Players,Year
0,Counter-Strike: Global Offensive,12,546614.19,950586,2021
1,Counter-Strike: Global Offensive,11,548161.67,935593,2021
2,Counter-Strike: Global Offensive,10,512435.85,864966,2021
3,Counter-Strike: Global Offensive,9,512350.92,942519,2021
4,Counter-Strike: Global Offensive,8,512081.96,802544,2021
...,...,...,...,...,...
5424,Hollow Knight,4,728.86,1305,2017
5425,Hollow Knight,3,1126.45,2027,2017
5426,Hollow Knight,2,192.30,2077,2017
5427,Hollow Knight,1,1.96,6,2017


마지막으로, 잘못 들어온 데이터가 있나 확인해보고 전처리 과정을 마무리한다.

In [11]:
data.describe()

,Month,Avg_Players,Peak_Players
count,5429.000000,5.429000e+03,5.429000e+03
mean,6.731811,3.769334e+04,7.035075e+04
std,3.458766,1.033032e+05,1.940765e+05
min,1.000000,0.000000e+00,0.000000e+00
25%,4.000000,6.509930e+03,1.226900e+04
50%,7.000000,1.215026e+04,2.314500e+04
75%,10.000000,2.430433e+04,4.663500e+04
max,12.000000,1.584887e+06,3.236027e+06


player수가 음수거나 그러진 않다. 또한, month가 1~12로 값이 잘 나왔다.

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5429 entries, 0 to 5428
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Game          5429 non-null   object 
 1   Month         5429 non-null   int64  
 2   Avg_Players   5429 non-null   float64
 3   Peak_Players  5429 non-null   int64  
 4   Year          5429 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 383.5+ KB


전체가 non-null이므로, data 정합성에는 큰 문제가 없을 것이다.
이제 이를 바탕으로 데이터 시각화를 진행하고, sql을 통해 데이터 추출등을 진행하여 비교할 정보를 뽑아서 진행하자.
pandas로도 쉽게 할 수 있지만, 시각화 툴 + google bigquery를 활용하는 것이 이번 프로젝트의 목표이다.

In [13]:
data.to_csv('prepared_steam_player_time_data.csv')